In [17]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."
assert TAVILY_API_KEY, "TAVILY_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")
print("Tavily Key loaded:", TAVILY_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)

# Example: Use Gemini
chain = prompt | gemini_llm
response = chain.invoke({"product": "smart home devices"})
print("Gemini response:", response)

# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
Tavily Key loaded: tvly-d...
Gemini response: content='Here are some creative names for a smart home device company, broken down by theme to inspire you:\n\n## Evocative & Modern\n\nThese names aim for a sense of wonder, sophistication, and futuristic living.\n\n1.  **AuraLogic:** Combines the ethereal "aura" with the practical "logic" of smart tech.\n2.  **Synapse Living:** Evokes the neural connections of the brain, suggesting an interconnected, intelligent home.\n3.  **Veridian Home:** "Veridian" suggests lushness, growth, and a thriving, intelligent environment.\n4.  **Zenith Dwell:** "Zenith" implies the peak or highest point of living, while "Dwell" is a classic term for home.\n5.  **LumenFlow:** "Lumen" (light) for clarity and vision, "Flow" for seamless operation and energy.\n6.  **EchoSphere:** Suggests responsiveness, an enveloping environment, and a cohesive system.\n7.  **CogniHab:** A blend of "Cognitive" (smart, thinking)

In [18]:
# Wrap LLM chain (PromptTemplate -> OpenAI) as a tool

# Reuse existing prompt and LLM (or switch to openai_llm2 if you prefer limits)
name_idea_chain = prompt | openai_llm

# Turn the chain into a tool
name_idea_tool = name_idea_chain.as_tool(
    name="name_idea_tool",
    description="Given a product description, generate a creative company name."
)

# Test the tool directly
tool_result = name_idea_tool.invoke({"product": "AI fitness wearables"})
print("name_idea_tool result:", tool_result)


name_idea_tool result: content='MindFit Technologies' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 22, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CpwV4P3RJI7VSV1AiW3SgTGgmY4dq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b4b58-5003-7322-9b8a-9ae8eb928676-0' usage_metadata={'input_tokens': 22, 'output_tokens': 3, 'total_tokens': 25, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [19]:
support_prompt = PromptTemplate(
    input_variables=["issue"],
    template=(
        "You are a customer support agent for a food delivery app.\n"
        "Customer issue:\n\"\"\"{issue}\"\"\"\n\n"
        "1) Write a short, polite reply (3–4 sentences).\n"
        "2) At the end, add: Priority: Low / Medium / High.\n"
    ),
)

# Chain: prompt -> OpenAI LLM
support_chain = support_prompt | openai_llm

# Wrap the chain as a tool
support_tool = support_chain.as_tool(
    name="support_reply",
    description="Given a customer issue, draft a reply and assign a priority."
)

# ---- Run the chain directly with invoke (recommended) ----
issue_text = (
    "My order arrived 45 minutes late and the food was already cold. "
    "This keeps happening with your service in my area."
)

# ---- Run the wrapped tool (tools always take a dict) ----
print("\n=== TOOL .invoke() ===")
tool_result = support_tool.invoke({"issue": issue_text})
print(tool_result)



=== TOOL .invoke() ===
content='I apologize for the inconvenience you experienced with your recent order. We strive to deliver food promptly and ensure it arrives hot. I will escalate this issue to our operations team to investigate and improve our service in your area. Priority: High.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 77, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CpwV4NsuqhFv9WWDZMICQKmGbnK3J', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b4b58-5366-7970-bd9c-13037d34e0a8-0' usage_metadata={'input_tokens': 77, 'output_tokens': 47, 'total_tokens': 124, 'input_token_details': {'au

In [20]:
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate

# ---------- 1) Define function tools ----------

@tool
def fahrenheit_to_celsius(fahrenheit: float) -> float:
    """Convert temperature in Fahrenheit to Celsius."""
    return round((fahrenheit - 32) * 5 / 9, 1)


@tool
def max_hotel_nights(budget_usd: float, price_per_night_usd: float) -> int:
    """Given a budget and price per night, return how many full nights you can afford."""
    if price_per_night_usd <= 0:
        raise ValueError("price_per_night_usd must be > 0")
    return int(budget_usd // price_per_night_usd)


# ---------- 2) LLM chain wrapped as a tool ----------

city_guide_prompt = PromptTemplate(
    input_variables=["city", "days"],
    template=(
        "You are a friendly local guide.\n"
        "The user will visit {city} for {days} days.\n"
        "Give 3 concise bullet points with must‑see places and food suggestions."
    ),
)

city_guide_chain = city_guide_prompt | openai_llm

city_guide_tool = city_guide_chain.as_tool(
    name="city_guide",
    description="Given a city and number of days, suggest must‑see places and food."
)

# ---------- 3) Use each tool directly with .invoke() ----------

print("=== fahrenheit_to_celsius tool ===")
print(
    fahrenheit_to_celsius.invoke({"fahrenheit": 86})
)

print("\n=== max_hotel_nights tool ===")
print(
    max_hotel_nights.invoke({"budget_usd": 800, "price_per_night_usd": 120})
)

print("\n=== city_guide_tool (LLM chain as tool) ===")
print(
    city_guide_tool.invoke({"city": "Tokyo", "days": "4"})
)

# ---------- 4) Bind MULTIPLE tools to the LLM, then .invoke() ----------

travel_tools = [fahrenheit_to_celsius, max_hotel_nights, city_guide_tool]

assistant_with_travel_tools = openai_llm.bind_tools(travel_tools)

user_query = (
    "I'm planning a 4‑day trip to Tokyo with a hotel budget of 800 USD and "
    "average price 120 USD per night. "
    "How many nights can I afford, and give me local tips. "
    "Also, if it's 86 F there, what is that in Celsius?"
)

print("\n=== assistant_with_travel_tools.invoke(...) ===")
response = assistant_with_travel_tools.invoke(user_query)
print(response)

=== fahrenheit_to_celsius tool ===
30.0

=== max_hotel_nights tool ===
6

=== city_guide_tool (LLM chain as tool) ===
content='Must-see places:\n- Visit the historic Senso-ji Temple in Asakusa\n- Explore the bustling streets of Shibuya and cross the famous Shibuya Crossing\n- Take a stroll through the beautiful gardens of the Imperial Palace\n\nFood suggestions:\n- Try some delicious sushi at the Tsukiji Fish Market\n- Indulge in a bowl of authentic ramen at Ichiran Ramen\n- Sample some traditional Japanese street food at the Tsukiji Outer Market' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 39, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'cha